This notebook will be used for calculating the power spectrum to be used in plotting for every epochs.

* Sample a lot of 64x64x64 cubes from the real 2048x2048x2048 cube
* Calculate power spectrum for each of the sample cubes
* Calculate mean and stddev for each k
* Save both as pickle to load later on

In [1]:
import graphviz
import argparse
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.utils as vutils
from torch.autograd import Variable, grad
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import os
import h5py
import timeit
import time
import json
from scipy import stats
import pickle as pkl
from os import listdir
from os.path import isfile, join
import re
import shutil


/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/h5py-2.7.1-py3.6-linux-x86_64.egg/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
run_in_jupyter = False
try:
    cfg = get_ipython().config 
    run_in_jupyter = True
except:
    run_in_jupyter = False
    pass

if run_in_jupyter:
    import matplotlib.pyplot as plt
    %matplotlib inline
else: 
    import matplotlib
    matplotlib.use('Agg')
    import matplotlib.pyplot as plt
    
print("Run in Jupyter = " + str(run_in_jupyter))

Run in Jupyter = True


In [3]:
# DONT MOVE THESE TO THE UPPER CELL!!!!!
import itertools
from matplotlib.ticker import MultipleLocator
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors
import h5py
import matplotlib as mpl
from pathlib import Path

## Notes

In [4]:
notes = "3D try"


### Training Options

In [5]:
run_mode = "continue"                       # training OR continue = continue if load another model is loaded and continued to train
continue_train_folder = "jupyter-output-128"    # "jupyter-output-552" # the folder where the previous run is for the saved model to be trained further
netD_iter_file = "netD_iter_16.pth"         # netD_iter_xx.pth file that contains the state dict under models/
netG_iter_file = "netG_iter_16.pth"         # netG_iter_xx.pth file that contains the state dict under models/

batch_size = 16                       # BATCH_SIZE: batch size for training
gpu_device = 0                              # GPU_DEVICE: gpu id (default 0)
nc = 1                # NC: number of channels in images
cube_size = 64       # for our dataset more like one edge of the subcube
lr = 1e-4               # LR: learning rate - default: 5e-5 (rmsprop) , 1e-4:adam
max_iter = 150         # MAX_ITER: max iteration for training

optimizer_choice = "adam"     # adam or rmsprop
adam_beta1 = 0.5     # default: 0.9    # coefficients used for computing running averages of gradient and its square 
adam_beta2 = 0.999     # default: 0.999
lr_decay = False               # Square root decay-> Just True or False        ||||Enter False or if True -> a numeric value

manual_seed = 1
sample_size_multiplier = 101 * 6
n_samples = batch_size * sample_size_multiplier      # on prince, number of samples to get from the training cube
Diter_1 = 100   # default: 100
Giter_1 = 1      # default: 1
Diter_2 = 1      # default: 5
Giter_2 = 1      # default: 1
if run_mode == "continue":
    gen_iterations_limit = 0  
else:
    gen_iterations_limit = 25 # default = 25
edge_sample = cube_size
edge_test = 512

mmd2_D_train_limit = False       # if True, if MMD2_D is less than 0, the generator training will be skipped
mmd2_D_skip_G_train = False

enable_gradient_penalty = True  # True = GP is used
lambda_gradpen = 1              # Juan = 10 | Demystifying MMD GANs = 1



In [6]:
assert n_samples > Diter_1, "The gen_iterations wont work properly!"

### Model Options

In [7]:
model_choice = "conv"                  # conv or con_fc
dimension_choice = "3D"                # 2D or 3D
nz = 128                           # hidden dimension channel

reconstruction_loss = False            # enable reconstruction loss or not
dist_ae = 'L2'                         # "L2" or "L1" -> Autoencoder reconstructruced cube loss choice,  "cos" doesnt work

repulsive_loss = False                   # False | lambda_repulsive ex: {False, 0.1, 0.7, 1.0}
bounded_rbf_kernel = True             # True or False

mmd_kernel = "rbf"                     # rbf , rbf_ratio, poly , linear, rq , rq_linear
sigma_list = [1]         # sigma for RBF Kernel MMD
alpha_list = [2,5,10,20,40,80]         # alpha list for RQ Kernel MMD #[1, 2, 5, 10, 20, 40, 80]    # [0.2,0.5,1.0,2.0,5.0]


weight_clip_enabled = False
left_clamp =  -0.01                    # default: -0.01
right_clamp = 0.01                     # default: 0.01

encoder_py = ""
decoder_py = ""

model_param_init = "normal"    # normal OR xavier (doesn't work right now)

"""
The explanations can be found in Ratio Matching MMD Nets (2018) in 
Equation 3.
"""
lambda_MMD = 1.0   # not used anywhere
lambda_AE_X = 8.0  # used in above calc only 
lambda_AE_Y = 8.0  # used in above calc only
lambda_rg = 0.1 #16.0   # errD = torch.sqrt(mmd2_D) + lambda_rg * one_side_errD \
                            # also for errG too

if not reconstruction_loss:
    lambda_AE_X = 0.0  # used in above calc only 
    lambda_AE_Y = 0.0  # used in above calc only
    

min_var_est = 1e-30 # 1e-30, default:1e-8


In [8]:
# if run_in_jupyter:
#     %run utils/power_spectrum_utils.py
# else:
from utils.power_spectrum_utils import *

### Plotting Options

In [9]:
viz_multiplier = 1e2                # the norm multiplier in the 3D visualization
scatter_size_magnitude = False      # change scatter point radius based on the value of the point 
if run_in_jupyter:
    plot_show_3d = True             # shows the 3d scatter plot
    plot_show_other = True
    plot_save_3d = True             # whether to save or not as png 
    plot_save_other = True
else:
    plot_show_3d = False            # shows the 3d scatter plot
    plot_show_other = False
    plot_save_3d = True             # whether to save or not as png 
    plot_save_other = True

### Saving Options

In [10]:
# if run_in_jupyter:
#     %run utils/logging_utils.py
# else:
from utils.logging_utils import *

In [11]:
root_dir = "./"  # this goes to 
data_dir = "../"
new_output_folder = get_output_folder(run_in_jupyter = run_in_jupyter)
# new_output_folder = "drive-output-XX"   # for batch processing
experiment = root_dir + "outputs/" + new_output_folder + "/"       # : output directory of saved models
# print(experiment)

model_save_folder = experiment + "model/"
redshift_fig_folder = experiment + "figures/"        # folder to save mmd & related plots
redshift_3dfig_folder = experiment + "3d_figures/"   # folder to save 3D plots
testing_folder = experiment + "testing/"   # folder to save 3D plots

save_model_every = 2               # (every x epoch) frequency to save the model


### Dataset Options

In [12]:
workers = 1        # WORKERS: number of threads to load data
redshift_info_folder = root_dir + "redshift_info/"   # save some info here as pickle to speed up processing
redshift_raw_file = "fields_z=5.0.hdf5"
# redshift_file = "redshift0_4th_root.h5"    # redshift cube to be used
                                        # standardized_no_shift_redshift0.h5
                                        # minmax_scale_01_redshift0.h5
                                        # minmax_scale_neg11_redshift0.h5
                                        # redshift0_4th_root.h5
                                        # redshift0_6th_root.h5
                                        # redshift0_8th_root.h5
                                        # redshift0_16th_root.h5
                                        # redshift0_4th_root_neg11.h5
root = 8 # should be an integer
inverse_transform = "log_scale_neg11"    # scale_01 / scale_neg11 / root / 
                                # root_scale_01 / root_scale_neg11
                                # log_scale_01 / log_scale_neg11
        

## Redshift Data Load

Loading the raw data instead of the transformed data because the transformations are going to be done on the fly.

In [13]:
# f = h5py.File(data_dir + redshift_file, 'r')
f = h5py.File(data_dir + redshift_raw_file, 'r')
print("File used for analysis = " + str(f.filename))
f = f['delta_HI']

File used for analysis = ../fields_z=5.0.hdf5


## Redshift Info Load

In [14]:
# create redshift info folder if it doesn't exist
if Path(redshift_info_folder).exists() == False:
    os.mkdir(redshift_info_folder)

In [15]:
# if run_in_jupyter:
#     %run utils/data_utils.py
# else:
from utils.data_utils import *

In [16]:
# min_cube,max_cube,mean_cube,stddev_cube = get_stats_cube(redshift_info_folder = redshift_info_folder,
#                                            redshift_file = redshift_file,
#                                            data_dir = data_dir)
min_cube,max_cube,mean_cube,stddev_cube = get_stats_cube(redshift_info_folder = redshift_info_folder,
                                           redshift_file = redshift_raw_file,
                                           data_dir = data_dir)

min_raw_cube,max_raw_cube,mean_raw_cube,stddev_raw_cube = get_stats_cube(redshift_info_folder = redshift_info_folder,
                                           redshift_file = redshift_raw_file,
                                           data_dir = data_dir)
print("\nTransformed  Data Summary Statistics:")
# print("File = " + str(redshift_file))
print("Min of data = " + str(min_cube))
print("Max of data = " + str(max_cube))
print("Mean of data = " + str(mean_cube))
print("Stddev of data = " + str(stddev_cube))

print("\nRaw Data Summary Statistics:")
print("File = " + str(redshift_raw_file))
print("Min of raw data = " + str(min_raw_cube))
print("Max of raw data = " + str(max_raw_cube))
print("Mean of raw data = " + str(mean_raw_cube))
print("Stddev of raw data = " + str(stddev_raw_cube))


Transformed  Data Summary Statistics:
Min of data = 0.0
Max of data = 4376932000.0
Mean of data = 14592.24
Stddev of data = 922711.56

Raw Data Summary Statistics:
File = fields_z=5.0.hdf5
Min of raw data = 0.0
Max of raw data = 4376932000.0
Mean of raw data = 14592.24
Stddev of raw data = 922711.56


In [17]:
from utils.plot_utils import *

In [18]:
from dataset import *

In [19]:
from test_3d_plot import *

## Dataset & DataLoader

In [20]:
# on prince
sampled_subcubes = HydrogenDataset(h5_file=redshift_raw_file,
                                    root_dir = data_dir,
                                    f = h5py.File(data_dir + redshift_raw_file, 'r')["delta_HI"],
                                    s_test = edge_test, 
                                    s_train = edge_sample,
                                    s_sample = edge_sample, 
                                    nsamples = n_samples,
                                   min_cube = min_cube,
                                  max_cube = max_cube,
                                  mean_cube = mean_cube,
                                  stddev_cube = stddev_cube,
                                   min_raw_cube = min_raw_cube,
                                  max_raw_cube = max_raw_cube,
                                  mean_raw_cube = mean_raw_cube,
                                  stddev_raw_cube = stddev_raw_cube,
                                  rotate_cubes = True,
                                   transform = False,
                                  root = root,
                                  dimensions = dimension_choice)



In [21]:
sampled_subcubes[0]

tensor([[[[6.5421e+00, 1.0711e+01, 3.1528e+00,  ..., 2.0225e+01,
           3.9625e+01, 1.4963e+01],
          [3.3278e+00, 6.1793e-01, 9.0984e+00,  ..., 1.3281e+01,
           2.4598e+01, 1.7787e+01],
          [3.4711e+00, 2.6696e+00, 3.0616e+00,  ..., 2.0637e+01,
           1.0468e+01, 1.8329e+01],
          ...,
          [3.4446e+00, 1.3478e+01, 1.4383e+01,  ..., 2.6585e+01,
           3.5941e+00, 5.8644e+00],
          [1.8003e+01, 6.8536e+00, 2.2541e+01,  ..., 4.5425e+01,
           5.0814e+00, 9.4452e+00],
          [1.9891e+01, 7.5610e+00, 1.7696e+01,  ..., 3.2916e+01,
           1.9948e+01, 1.9706e+01]],

         [[4.0020e+00, 2.7466e+00, 2.7559e+00,  ..., 5.0444e+01,
           6.5539e+01, 3.0063e+01],
          [1.4743e-01, 2.1656e+00, 1.5861e+00,  ..., 4.2084e+01,
           5.2978e+01, 2.6939e+01],
          [3.5366e-01, 1.1314e+01, 5.7064e+00,  ..., 3.8384e+01,
           3.6316e+01, 3.0298e+01],
          ...,
          [1.5088e+01, 2.0145e+01, 2.8516e+01,  ..., 1.1642

In [22]:
Pks = []

for i in range(5000):
    if i % 100 == 0: print(i)
    Pk_sample, k_sample = power_spectrum_np(cube = np.array(sampled_subcubes[0]), 
                                          mean_raw_cube = sampled_subcubes.mean_raw_val, 
                                          data_dim = "3D")
#     print("Pk: {}".format(Pk_sample))
#     print("k: {}".format(k_sample))
#     print(len(Pk_sample))
    Pks.append(Pk_sample)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [23]:
Pks = np.array(Pks)
Pks.shape

(5000, 31)

In [33]:
# mean
Pk_means = np.mean(a=Pks,axis=0)
# Pk_means

In [32]:
# stddev
Pk_std = np.std(a=Pks,axis=0)
# Pk_std

In [31]:
# 5th, 25th, 75th and 95th percentile
Pk_min = np.percentile(a=Pks,q=0,axis=0)
Pk_5th = np.percentile(a=Pks,q=5,axis=0)
Pk_25th = np.percentile(a=Pks,q=25,axis=0)
Pk_50th = np.percentile(a=Pks,q=50,axis=0)
Pk_75th = np.percentile(a=Pks,q=75,axis=0)
Pk_95th = np.percentile(a=Pks,q=95,axis=0)
Pk_max = np.percentile(a=Pks,q=100,axis=0)

# print(Pk_min)
# print(Pk_5th)
# print(Pk_25th)
# print(Pk_50th)
# print(Pk_75th)
# print(Pk_95th)
# print(Pk_max)

In [36]:
power_spectrum_real = {"Pk_means":Pk_means,
                       "Pk_std":Pk_std,
                       "Pk_min":Pk_min,
                       "Pk_5th":Pk_5th,
                       "Pk_25th":Pk_25th,
                       "Pk_50th":Pk_50th,
                       "Pk_75th":Pk_75th,
                       "Pk_95th":Pk_95th,
                       "Pk_max":Pk_max}
with open('power_spectrum/power_spectrum_real.pickle', 'wb') as handle:
    pkl.dump(power_spectrum_real, handle, protocol=pkl.HIGHEST_PROTOCOL)